In [1]:
import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.1.0'

In [3]:
from tensorflow.keras.datasets import boston_housing

(train_data,train_target),(test_data,test_target)=boston_housing.load_data()

In [4]:
train_data.shape

(404, 13)

In [5]:
train_data[0] #13個特徵

array([  1.23247,   0.     ,   8.14   ,   0.     ,   0.538  ,   6.142  ,
        91.7    ,   3.9769 ,   4.     , 307.     ,  21.     , 396.9    ,
        18.72   ])

In [6]:
train_target.shape

(404,)

In [7]:
train_target[0] #成交中位數

15.2

# 正規化
### 特徵值減去平均(mean)並除以標準差(std)
### 數值會以0為中心、以標準差為單位刻度

In [8]:
mean=train_data.mean(axis=0)
train_data-=mean

std=train_data.std(axis=0)
train_data/=std

test_data-=mean
test_data/=std

In [10]:
def net():
    model=keras.Sequential([
        keras.layers.Dense(64, activation='relu',input_shape=(train_data.shape[1], )),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1)
    ])
    
    model.compile(optimizer='rmsprop',loss='mse',metrics=['mae'])
    return model

# K-fold cross validation

In [11]:
import numpy as np
k=4
num_val_sample=len(train_data)//k
num_epochs=100
all_scores=[]

In [12]:
a=[1,2]
b=[3,4]
np.concatenate([a,b])

array([1, 2, 3, 4])

In [14]:
for i in range(k):
    print('processing fold #',i)
    val_data = train_data[i * num_val_sample: (i + 1) * num_val_sample] #驗證資料
    val_targets = train_target[i * num_val_sample: (i + 1) * num_val_sample]
    
    partial_train_data = np.concatenate( #k之外的資料當訓練
        [train_data[:i * num_val_sample],
         train_data[(i + 1) * num_val_sample:]],
        axis=0)
    partial_train_targets=np.concatenate(
        [train_target[:i*num_val_sample],
        train_target[(i+1)*num_val_sample:]],
        axis=0)
    
    model=net()
    model.fit(partial_train_data,partial_train_targets,epochs=num_epochs,batch_size=1,verbose=0)
    
    val_mse,val_mae=model.evaluate(val_data,val_targets,verbose=0)
    all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [15]:
all_scores

[2.1098008, 2.880605, 2.646421, 2.213063]

In [16]:
np.mean(all_scores)

2.4624724